In [ ]:
# https://scikit-learn.org/stable/modules/multiclass.html

In [1]:
# Import libraries
import pandas as pd
import numpy as np
pd.set_option('max_colwidth', 400)
import requests
import matplotlib.pyplot as plt
import json
import re
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [2]:
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [3]:

salaries_df = pd.read_csv('salaries_cleaned.csv')

In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [4]:
salaries_df.head()

,Unnamed: 0,timestamp,date,company,level,title,totalyearlycompensation,location,latitude,longitude,yearsofexperience,yearsatcompany,basesalary,stockgrantvalue,bonus,gender,Race,Education
0,0,2017-06-07 11:33:00,2017-06-07 00:00:00,Oracle,L3,Product Manager,127000,"Redwood City, CA",37.486324,-122.232523,1.5,1.5,107000,20000,10000,Unknown,Unknown,Unknown
1,1,2017-06-10 17:11:00,2017-06-10 00:00:00,eBay,SE 2,Software Engineer,100000,"San Francisco, CA",37.779026,-122.419906,5.0,3.0,0,0,0,Unknown,Unknown,Unknown
2,2,2017-06-11 14:53:00,2017-06-11 00:00:00,Amazon,L7,Product Manager,310000,"Seattle, WA",47.603832,-122.330062,8.0,0.0,155000,0,0,Unknown,Unknown,Unknown
3,3,2017-06-17 00:23:00,2017-06-17 00:00:00,Apple,M1,Software Engineering Manager,372000,"Sunnyvale, CA",37.368830,-122.036349,7.0,5.0,157000,180000,35000,Unknown,Unknown,Unknown
4,4,2017-06-20 10:58:00,2017-06-20 00:00:00,Microsoft,60,Software Engineer,157000,"Mountain View, CA",37.389389,-122.083210,5.0,3.0,0,0,0,Unknown,Unknown,Unknown


In [5]:
salaries_df.columns

Index(['Unnamed: 0', 'timestamp', 'date', 'company', 'level', 'title',
       'totalyearlycompensation', 'location', 'latitude', 'longitude',
       'yearsofexperience', 'yearsatcompany', 'basesalary', 'stockgrantvalue',
       'bonus', 'gender', 'Race', 'Education'],
      dtype='object')

,title
Human Resources,364
Recruiter,451
Sales,461
Mechanical Engineer,490
Marketing,710
Business Analyst,885
Management Consultant,976
Solution Architect,1157
Technical Program Manager,1381
Product Designer,1516


In [6]:
salaries_df['company'].value_counts()

Amazon                  8197
Microsoft               5251
Google                  4365
Facebook                3004
Apple                   2045
                        ... 
PayTm                      1
Willis towers watson       1
INTEL corporation          1
SONY                       1
SalesForce                 1
Name: company, Length: 1185, dtype: int64

In [7]:
val=salaries_df['company'].value_counts()
print(val[val>25])

Amazon       8197
Microsoft    5251
Google       4365
Facebook     3004
Apple        2045
             ... 
Glassdoor      26
HBO            26
Fortinet       26
Cohesity       26
Sony           26
Name: company, Length: 263, dtype: int64


In [8]:
#companies over 25 jobs
comapanies_to_replace=[]
company=salaries_df['company'].value_counts()
company=pd.DataFrame(company)
company.head(20)

,company
Amazon,8197
Microsoft,5251
Google,4365
Facebook,3004
Apple,2045
Oracle,1143
Salesforce,1064
Intel,974
Cisco,939
IBM,917


In [9]:
comapanies_to_replace.append(company[company.company<15].index)

AttributeError: 'list' object has no attribute 'value_counts'

In [10]:
#Replaced with other for all companies besides the top 20
for x in comapanies_to_replace:
    salaries_df['company'] = salaries_df['company'].replace(x,"Other")
    
# Check to make sure binning was successful
salaries_df['company'].value_counts()

Amazon         8197
Microsoft      5251
Other          5117
Google         4365
Facebook       3004
               ... 
Genesys          15
Gartner          15
WarnerMedia      15
WePay            15
BAE Systems      15
Name: company, Length: 395, dtype: int64

In [11]:
year_df=salaries_df['date'].str.split('-', expand=True)

In [12]:
salaries_df['date']=year_df[0]

In [13]:
salaries_df['month']=year_df[1]

In [14]:
salaries_df

,Unnamed: 0,timestamp,date,company,level,title,totalyearlycompensation,location,latitude,longitude,yearsofexperience,yearsatcompany,basesalary,stockgrantvalue,bonus,gender,Race,Education,month
0,0,2017-06-07 11:33:00,2017,Oracle,L3,Product Manager,127000,"Redwood City, CA",37.486324,-122.232523,1.5,1.5,107000,20000,10000,Unknown,Unknown,Unknown,06
1,1,2017-06-10 17:11:00,2017,eBay,SE 2,Software Engineer,100000,"San Francisco, CA",37.779026,-122.419906,5.0,3.0,0,0,0,Unknown,Unknown,Unknown,06
2,2,2017-06-11 14:53:00,2017,Amazon,L7,Product Manager,310000,"Seattle, WA",47.603832,-122.330062,8.0,0.0,155000,0,0,Unknown,Unknown,Unknown,06
3,3,2017-06-17 00:23:00,2017,Apple,M1,Software Engineering Manager,372000,"Sunnyvale, CA",37.368830,-122.036349,7.0,5.0,157000,180000,35000,Unknown,Unknown,Unknown,06
4,4,2017-06-20 10:58:00,2017,Microsoft,60,Software Engineer,157000,"Mountain View, CA",37.389389,-122.083210,5.0,3.0,0,0,0,Unknown,Unknown,Unknown,06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62637,62637,2018-09-09 11:52:00,2018,Google,T4,Software Engineer,327000,"Seattle, WA",47.603832,-122.330062,10.0,1.0,155000,150000,22000,Unknown,Unknown,Unknown,09
62638,62638,2018-09-13 08:23:00,2018,Microsoft,62,Software Engineer,237000,"Redmond, WA",47.669414,-122.123877,2.0,2.0,146900,73200,16000,Unknown,Unknown,Unknown,09
62639,62639,2018-09-13 14:35:00,2018,Other,63,Software Engineer,220000,"Seattle, WA",47.603832,-122.330062,14.0,12.0,157000,25000,20000,Unknown,Unknown,Unknown,09
62640,62640,2018-09-16 16:10:00,2018,Salesforce,Lead MTS,Software Engineer,280000,"San Francisco, CA",37.779026,-122.419906,8.0,4.0,194688,57000,29000,Unknown,Unknown,Unknown,09


In [15]:
salaries_df['level'].value_counts()

L4                              5014
L5                              4871
L3                              3337
L6                              2871
Senior Software Engineer        1443
                                ... 
Gr 7                               1
Senior BA                          1
Consulting Analyst                 1
Senior Technical Director II       1
Bioinformatics Scientist II        1
Name: level, Length: 2916, dtype: int64

In [16]:
#companies over 25 jobs
levels_to_replace=[]
levels=salaries_df['level'].value_counts()
levels=pd.DataFrame(levels)
levels.head(20)

,level
L4,5014
L5,4871
L3,3337
L6,2871
Senior Software Engineer,1443
L2,1163
Senior,1079
L7,919
L1,765
62,764


In [17]:
levels_to_replace.append(levels[levels.level<15].index)

In [18]:
for x in levels_to_replace:
    salaries_df['level'] = salaries_df['level'].replace(x,"Other")
    
# Check to make sure binning was successful
salaries_df['level'].value_counts()

Other                            5668
L4                               5014
L5                               4871
L3                               3337
L6                               2871
                                 ... 
New grad                           15
Engineer I                         15
Only one level across Netflix      15
Grade 14                           15
PR3                                15
Name: level, Length: 326, dtype: int64

In [19]:
reduced_df = salaries_df[[ 'company', 'date', 'title', 'totalyearlycompensation', 'gender', 'level',
       'yearsofexperience', 'yearsatcompany', 'latitude', 'longitude', 'month', 'bonus', 'stockgrantvalue']]

In [20]:
reduced_df


,company,date,title,totalyearlycompensation,gender,level,yearsofexperience,yearsatcompany,latitude,longitude,month,bonus,stockgrantvalue
0,Oracle,2017,Product Manager,127000,Unknown,L3,1.5,1.5,37.486324,-122.232523,06,10000,20000
1,eBay,2017,Software Engineer,100000,Unknown,SE 2,5.0,3.0,37.779026,-122.419906,06,0,0
2,Amazon,2017,Product Manager,310000,Unknown,L7,8.0,0.0,47.603832,-122.330062,06,0,0
3,Apple,2017,Software Engineering Manager,372000,Unknown,M1,7.0,5.0,37.368830,-122.036349,06,35000,180000
4,Microsoft,2017,Software Engineer,157000,Unknown,60,5.0,3.0,37.389389,-122.083210,06,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
62637,Google,2018,Software Engineer,327000,Unknown,T4,10.0,1.0,47.603832,-122.330062,09,22000,150000
62638,Microsoft,2018,Software Engineer,237000,Unknown,62,2.0,2.0,47.669414,-122.123877,09,16000,73200
62639,Other,2018,Software Engineer,220000,Unknown,63,14.0,12.0,47.603832,-122.330062,09,20000,25000
62640,Salesforce,2018,Software Engineer,280000,Unknown,Lead MTS,8.0,4.0,37.779026,-122.419906,09,29000,57000


In [21]:
reduced_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62642 entries, 0 to 62641
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   company                  62637 non-null  object 
 1   date                     62642 non-null  object 
 2   title                    62642 non-null  object 
 3   totalyearlycompensation  62642 non-null  int64  
 4   gender                   62642 non-null  object 
 5   level                    62523 non-null  object 
 6   yearsofexperience        62642 non-null  float64
 7   yearsatcompany           62642 non-null  float64
 8   latitude                 62642 non-null  float64
 9   longitude                62642 non-null  float64
 10  month                    62642 non-null  object 
 11  bonus                    62642 non-null  int64  
 12  stockgrantvalue          62642 non-null  int64  
dtypes: float64(4), int64(3), object(6)
memory usage: 6.2+ MB


In [22]:

reduced_df['date']=reduced_df['date'].astype(int)
reduced_df['month']=reduced_df['month'].astype(int)
reduced_df

,company,date,title,totalyearlycompensation,gender,level,yearsofexperience,yearsatcompany,latitude,longitude,month,bonus,stockgrantvalue
0,Oracle,2017,Product Manager,127000,Unknown,L3,1.5,1.5,37.486324,-122.232523,6,10000,20000
1,eBay,2017,Software Engineer,100000,Unknown,SE 2,5.0,3.0,37.779026,-122.419906,6,0,0
2,Amazon,2017,Product Manager,310000,Unknown,L7,8.0,0.0,47.603832,-122.330062,6,0,0
3,Apple,2017,Software Engineering Manager,372000,Unknown,M1,7.0,5.0,37.368830,-122.036349,6,35000,180000
4,Microsoft,2017,Software Engineer,157000,Unknown,60,5.0,3.0,37.389389,-122.083210,6,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
62637,Google,2018,Software Engineer,327000,Unknown,T4,10.0,1.0,47.603832,-122.330062,9,22000,150000
62638,Microsoft,2018,Software Engineer,237000,Unknown,62,2.0,2.0,47.669414,-122.123877,9,16000,73200
62639,Other,2018,Software Engineer,220000,Unknown,63,14.0,12.0,47.603832,-122.330062,9,20000,25000
62640,Salesforce,2018,Software Engineer,280000,Unknown,Lead MTS,8.0,4.0,37.779026,-122.419906,9,29000,57000


In [23]:
reduced_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62642 entries, 0 to 62641
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   company                  62637 non-null  object 
 1   date                     62642 non-null  int32  
 2   title                    62642 non-null  object 
 3   totalyearlycompensation  62642 non-null  int64  
 4   gender                   62642 non-null  object 
 5   level                    62523 non-null  object 
 6   yearsofexperience        62642 non-null  float64
 7   yearsatcompany           62642 non-null  float64
 8   latitude                 62642 non-null  float64
 9   longitude                62642 non-null  float64
 10  month                    62642 non-null  int32  
 11  bonus                    62642 non-null  int64  
 12  stockgrantvalue          62642 non-null  int64  
dtypes: float64(4), int32(2), int64(3), object(4)
memory usage: 5.7+ MB


In [24]:
dummies_df = pd.get_dummies(reduced_df)
dummies_df.head()

,date,totalyearlycompensation,yearsofexperience,yearsatcompany,latitude,longitude,month,bonus,stockgrantvalue,company_3M,...,level_Technical Leader 2,level_VP,level_Vice President,level_l1,level_l2,level_l3,level_l4,level_l5,level_l6,level_senior
0,2017,127000,1.5,1.5,37.486324,-122.232523,6,10000,20000,0,...,0,0,0,0,0,0,0,0,0,0
1,2017,100000,5.0,3.0,37.779026,-122.419906,6,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2017,310000,8.0,0.0,47.603832,-122.330062,6,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2017,372000,7.0,5.0,37.368830,-122.036349,6,35000,180000,0,...,0,0,0,0,0,0,0,0,0,0
4,2017,157000,5.0,3.0,37.389389,-122.083210,6,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
dummies_df.to_csv("dummies_table.csv")

In [26]:
dummies_df.columns

Index(['date', 'totalyearlycompensation', 'yearsofexperience',
       'yearsatcompany', 'latitude', 'longitude', 'month', 'bonus',
       'stockgrantvalue', 'company_3M',
       ...
       'level_Technical Leader 2', 'level_VP', 'level_Vice President',
       'level_l1', 'level_l2', 'level_l3', 'level_l4', 'level_l5', 'level_l6',
       'level_senior'],
      dtype='object', length=749)

In [27]:
# Split our preprocessed data into our features and target arrays
X = dummies_df.drop(columns=["totalyearlycompensation"]).values
y = dummies_df["totalyearlycompensation"].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [28]:
# # Split our preprocessed data into our features and target arrays
# X = reduced_df.drop(columns=["totalyearlycompensation"]).values
# y = reduced_df["totalyearlycompensation"].values

# # Split the preprocessed data into a training and testing dataset
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [29]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [30]:
X.shape

(62642, 748)

In [31]:
#linear regression
# Import required libraries
import numpy as np
import pandas as pd

from pathlib import Path
from sklearn.linear_model import LinearRegression

In [32]:
#random forest
import pandas as pd
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
%matplotlib inline

In [33]:
#https://stackoverflow.com/questions/52648383/how-to-get-coefficients-and-feature-importances-from-multioutputregressor
#rf_model = MultiOutputRegressor(RandomForestRegressor(n_estimators=500, random_state=78, max_depth=20))

In [34]:
rf_model = RandomForestRegressor(n_estimators=500, random_state=78, max_depth=20, n_jobs=-1, verbose=3, oob_score=True)
#rf_model = RandomForestRegressor(n_estimators=500, random_state=7, max_depth=10)

In [ ]:
rf_model = rf_model.fit(X_train_scaled, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 500
building tree 2 of 500
building tree 3 of 500
building tree 4 of 500
building tree 5 of 500
building tree 6 of 500
building tree 7 of 500
building tree 8 of 500
building tree 9 of 500
building tree 10 of 500
building tree 11 of 500
building tree 12 of 500
building tree 13 of 500
building tree 14 of 500
building tree 15 of 500
building tree 16 of 500
building tree 17 of 500
building tree 18 of 500
building tree 19 of 500
building tree 20 of 500
building tree 21 of 500
building tree 22 of 500
building tree 23 of 500
building tree 24 of 500


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   19.6s


building tree 25 of 500
building tree 26 of 500
building tree 27 of 500
building tree 28 of 500
building tree 29 of 500
building tree 30 of 500
building tree 31 of 500
building tree 32 of 500


In [ ]:
predictions = rf_model.predict(X_test_scaled)

In [ ]:
plt.scatter( predictions, y_test)
plt.xlabel("predictions")
plt.ylabel("y_test")
plt.show()

In [ ]:
acc_score = rf_model.score(X_test_scaled, y_test)
acc_score
#0.7402413883001011 with bonus
#0.8639637152876211 with stock
#0.8623347451720718 with updated company list

In [ ]:
# import pickle
# #save model
# s = pickle.dumps(rf_model)
# #load model
# clf2 = pickle.loads(s)


In [ ]:
#save the model 
#https://scikit-learn.org/stable/model_persistence.html
from joblib import dump, load
dump(rf_model, 'salary_ml_model.joblib') 

In [50]:
# have to run inputs throught get dummies_df, maybe set all 700+ values to zero and then just change individual values based on the input
#maybe do throught for loop 
#scales the data
#then take the scaled inputs 